In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# download ethereum data from yfinance
eth = yf.download('AAPL', period='max')

# convert to pandas dataframe
eth = pd.DataFrame(eth)

eth.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099874,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094663,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087715,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089886,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092492,73449600


In [3]:
def delete_consecutive_rows(df):
    # create a new dataframe to store the cleaned data
    cleaned_df = pd.DataFrame()
    
    # set the previous day to "none"
    prev_day = "none"
    
    # iterate through the rows of the dataframe
    for index, row in df.iterrows():
        # get the current day
        day = index.weekday()
        if day == 4:
            day = "friday"
        elif day == 0:
            day = "monday"
        else:
            continue
        
        # check if the current day is the same as the previous day
        if day != prev_day:
            # add the current row to the cleaned dataframe
            cleaned_df = cleaned_df.append(row)
            # update the previous day
            prev_day = day
    return cleaned_df


In [197]:
mask = (eth.index.weekday == 4) | (eth.index.weekday == 0)

eth = eth[mask]
eth = delete_consecutive_rows(eth)

/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(row)
/tmp/ipykernel_12273/1272178690.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

In [4]:
# drop every date that is not friday or monday
# Create a boolean mask that filters out the desired dates
mask_friday = (eth.index.weekday == 4)
mask_monday = (eth.index.weekday == 0)

# use the boolean mask to filter the dataframe and keep only friday and monday
eth_x = eth[mask_friday]
eth_y = eth[mask_monday]

print(eth_x.head())
print(eth_y.head())



                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
1980-12-12  0.128348  0.128906  0.128348  0.128348   0.099874  469033600
1980-12-19  0.126116  0.126674  0.126116  0.126116   0.098137   48630400
1980-12-26  0.158482  0.159040  0.158482  0.158482   0.123323   55574400
1981-01-02  0.154018  0.155134  0.154018  0.154018   0.119849   21660800
1981-01-09  0.142299  0.142857  0.142299  0.142299   0.110730   21504000
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
1980-12-15  0.122210  0.122210  0.121652  0.121652   0.094663  175884800
1980-12-22  0.132254  0.132813  0.132254  0.132254   0.102913   37363200
1980-12-29  0.160714  0.161272  0.160714  0.160714   0.125059   93161600
1981-01-05  0.151228  0.151228  0.150670  0.150670   0.117244   35728000
1981-01-12  0.142299  0.142299  0.141183  0.141183 

In [5]:
# change eth_x to percent change
eth_x = eth_x.pct_change()

# change inf to 0
eth_x = eth_x.replace([np.inf, -np.inf], 0)

# drop na
eth_x = eth_x.dropna()

# drop close
eth_x = eth_x.drop(['Close'], axis=1)

In [6]:
eth_x

,Open,High,Low,Adj Close,Volume
Date,,,,,
1980-12-19,-0.017390,-0.017315,-0.017390,-0.017390,-0.896318
1980-12-26,0.256637,0.255506,0.256637,0.256637,0.142791
1981-01-02,-0.028167,-0.024560,-0.028167,-0.028167,-0.610238
1981-01-09,-0.076089,-0.079138,-0.076089,-0.076089,-0.007239
1981-01-16,-0.023528,-0.027342,-0.027449,-0.027449,-0.377083
...,...,...,...,...,...
2022-12-23,-0.042212,-0.037995,-0.030584,-0.019701,-0.601548
2022-12-30,-0.019172,-0.018653,-0.017047,-0.014637,0.205997
2023-01-06,-0.018690,0.002616,-0.019933,-0.002386,0.139370


In [7]:
# save the eth_x index
eth_x_index = eth_x.index
print(eth_x_index)
print(len(eth_x_index))

# save the eth_x columns
eth_x_columns = eth_x.columns

# scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
eth_x = scaler.fit_transform(eth_x)

# put eth_x to pandas dataframe
eth_x = pd.DataFrame(eth_x, columns=eth_x_columns, index=eth_x_index)

eth_x

DatetimeIndex(['1980-12-19', '1980-12-26', '1981-01-02', '1981-01-09',
               '1981-01-16', '1981-01-23', '1981-01-30', '1981-02-06',
               '1981-02-13', '1981-02-20',
               ...
               '2022-11-18', '2022-11-25', '2022-12-02', '2022-12-09',
               '2022-12-16', '2022-12-23', '2022-12-30', '2023-01-06',
               '2023-01-13', '2023-01-20'],
              dtype='datetime64[ns]', name='Date', length=2120, freq=None)
2120


,Open,High,Low,Adj Close,Volume
Date,,,,,
1980-12-19,0.414988,0.464087,0.484783,0.541160,0.003120
1980-12-26,0.684232,0.758779,0.764390,0.844293,0.055356
1981-01-02,0.404400,0.456262,0.473787,0.529238,0.017501
1981-01-09,0.357315,0.397308,0.424890,0.476226,0.047814
1981-01-16,0.408958,0.453257,0.474520,0.530032,0.029222
...,...,...,...,...,...
2022-12-23,0.390600,0.441750,0.471321,0.538603,0.017938
2022-12-30,0.413238,0.462642,0.485133,0.544205,0.058533
2023-01-06,0.413711,0.485616,0.482189,0.557757,0.055184


In [8]:
eth_y

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094663,175884800
1980-12-22,0.132254,0.132813,0.132254,0.132254,0.102913,37363200
1980-12-29,0.160714,0.161272,0.160714,0.160714,0.125059,93161600
1981-01-05,0.151228,0.151228,0.150670,0.150670,0.117244,35728000
1981-01-12,0.142299,0.142299,0.141183,0.141183,0.109861,23699200
...,...,...,...,...,...,...
2022-11-28,145.139999,146.639999,143.380005,144.220001,144.220001,69246000
2022-12-05,147.770004,150.919998,145.770004,146.630005,146.630005,68826400
2022-12-12,142.699997,144.500000,141.059998,144.490005,144.490005,70462700


In [9]:

# drop from y all other values than open
eth_y = eth_y.drop(['High', 'Low', 'Close', 'Volume', "Adj Close"], axis=1)

# remove datetimeindex from eth_y
eth_y = eth_y.reset_index()

# rename eth_y column to target_open
eth_y = eth_y.rename(columns={'Open': 'target_open'})

# set eth_y's index the index from eth_x
eth_y = eth_y.set_index(eth_x.index)

# join eth_x and eth_y
eth = eth_x.join(eth_y)

eth['target_open'] = eth['target_open'].shift(1)

# drop date from eth
eth = eth.drop(['index'], axis=1)

# drop na
eth = eth.dropna()

eth

ValueError: Length mismatch: Expected 2006 rows, received array of length 2120

In [203]:
# split into x and y, y is target_open
x = eth.drop(['target_open'], axis=1)
y = eth['target_open']

# split into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [204]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# create DMatrix
dtrain = xgb.DMatrix(x_train, label=y_train)

# create parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'eta': [0.01, 0.1, 0.3, 0.5, 0.7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'lambda': [1, 2, 4],
    'alpha': [0, 1, 2],
    'objective': ['reg:squarederror'],
    'eval_metric': ['rmse']
}

# create XGBoost model
xgb_model = xgb.XGBRegressor()

# create GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

# fit to data
grid_search.fit(x_train, y_train)

# view best parameters
print(grid_search.best_params_)

# create DMatrix for test data
dtest = xgb.DMatrix(x_test)

# predict using best parameters
y_pred = grid_search.predict(x_test)

# plot, y_test index to match y_pred
plt.plot(y_test.index, y_pred, label='Predicted')
plt.plot(y_test.index, y_test, label='Actual')
plt.legend()
plt.show()

# calculate rmse
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))


KeyboardInterrupt: 